In [150]:
import pandas as pd
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import joblib

In [339]:
p = "pool_0010"
s = "schedule_0010"

pool, schedule = pd.read_csv(f'scheduling_problems/pools/{p}.csv',dtype={'employee_id':'str'}), \
                 pd.read_csv(f'scheduling_problems/schedules/{s}.csv',dtype={'reward_step':'str'})

sdf = schedule.copy()
schedule['shift_day_of_week'] = schedule['shift_day_of_week'].replace(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'],[1, 2, 3, 4, 5])

schedule['shift_type'] = schedule['shift_type'].replace(['Morning', 'Evening'],[1, 2])

schedule

,shift_id,shift_day_of_week,shift_type
0,0,1,1
1,1,2,1
2,2,2,2
3,3,3,1
4,4,3,2
5,5,4,1
6,6,5,1


In [341]:
sfEncodings = joblib.load('shiftFeatureEncoding.joblib')

shifts = pd.get_dummies(schedule[['shift_id']],drop_first=True)
sfEncoded =  sfEncodings.transform(schedule[['shift_day_of_week','shift_type']])
shift_features = pd.DataFrame(sfEncoded, columns=sfEncodings.get_feature_names_out())

schedule = pd.merge(shifts, shift_features, left_index=True, right_index=True)

shift_features = schedule.shape[1]

for i in pd.get_dummies(pool).columns.to_list():
    schedule[i] = 0

schedule

,shift_id,shift_day_of_week_2,shift_day_of_week_3,shift_day_of_week_4,shift_day_of_week_5,shift_type_2,employee_id_550035,employee_id_804870,employee_id_847472,employee_id_851241
0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
1,1,1.0,0.0,0.0,0.0,0.0,0,0,0,0
2,2,1.0,0.0,0.0,0.0,1.0,0,0,0,0
3,3,0.0,1.0,0.0,0.0,0.0,0,0,0,0
4,4,0.0,1.0,0.0,0.0,1.0,0,0,0,0
5,5,0.0,0.0,1.0,0.0,0.0,0,0,0,0
6,6,0.0,0.0,0.0,1.0,0.0,0,0,0,0


In [342]:
num_shifts = len(schedule)
count_workers=len(pool)

state = schedule.to_numpy()
state

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [2., 1., 0., 0., 0., 1., 0., 0., 0., 0.],
       [3., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [4., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [5., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [6., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [355]:
def evaluateSchedule(state):
    """Check a completed schedule for constraint violations.
    For each employee, apply sliding window to compare successive shifts.
    If an employee is assigned to b2b shifts, look up the relevant features.
    If the shifts are on the same day, record a constraint violation
    If the shifts are on successive days, evening then morning, records a constraint  violation.
    Else, no constraint violation.

    :param state: state matrix
    :type state: numpy.array
    :return: count of constraint violations
    :rtype: int
    """
    count_b2b_violation = 0

    for i in range(count_workers):
        # using sliding window to compare successive shifts
        # checks = a list of pairwise binary shift assignment feature, from last - first
        checks = sliding_window_view(state[:,shift_features+i], 2)[::-1]
        #print(checks)
        # for each check
        for j,k in enumerate(checks):
            shift_id = abs(j - len(checks))-1
            # check for b2b shifts
            # 1 = assigned, 0 = not assigned
            if sum(k) > 1:
                #print(f"employee:{i}, shift:{shift_id},{k}")
                # get features for b2b shifts
                shift_feats = state[shift_id:shift_id+2,num_shifts-1:shift_features]

                # just the features for day of week
                day_feats_1 = state[shift_id,num_shifts-1:shift_features-1]
                day_feats_2 = state[shift_id+1,num_shifts-1:shift_features-1]
                day1 = [np.where(day_feats_1==1)[0].item() + 2 if np.where(day_feats_1==1)[0].size != 0 else 1][0]
                day2 = [np.where(day_feats_2==1)[0].item() + 2 if np.where(day_feats_2==1)[0].size != 0 else 1][0]
                
                # shifts are on the same day = violation
                if day1 == day2:
                    count_b2b_violation += 1
                    print(f"shift:{shift_id+1},employee:{i}, constraint1 violated")
                
                # if shifts are on successive days, evening -> morning = violation
                if day2 == day1+1:
                    # if shift 1 type = evening and shift 2 type = morning, record violation
                    count_b2b_violation += [1 if shift_feats[:,4][0] == 1 and shift_feats[:,4][1] == 0 else 0][0]
                    if [1 if shift_feats[:,4][0] == 1 and shift_feats[:,4][1] == 0 else 0][0] == 1:
                        print(f"shift:{shift_id+1},employee:{i}, constraint2 violated")


    return count_b2b_violation

In [354]:
def evaluateStep(state,reward_step):
    """Check the last 2 shift assignments for constraint violations.
    For each employee, compare successive shifts.
    If an employee is assigned to b2b shifts, look up the relevant features.
    If the shifts are on the same day, record a constraint violation
    If the shifts are on successive days, evening then morning, records a constraint  violation.
    Else, no constraint violation.

    :param state: state matrix
    :type state: numpy.array
    :return: count of constraint violations
    :rtype: int
    """
    count_b2b_violation = 0

    for i in range(count_workers):
        assignments = state[reward_step-1:reward_step+1,shift_features:][:,i]
        if sum(assignments) > 1:
            shift_feats = state[reward_step-1:reward_step+1,num_shifts-1:shift_features]

            # just the features for day of week
            day_feats_1 = state[reward_step-1,num_shifts-1:shift_features-1]
            day_feats_2 = state[reward_step,num_shifts-1:shift_features-1]
            day1 = [np.where(day_feats_1==1)[0].item() + 2 if np.where(day_feats_1==1)[0].size != 0 else 1][0]
            day2 = [np.where(day_feats_2==1)[0].item() + 2 if np.where(day_feats_2==1)[0].size != 0 else 1][0]
            
            # shifts are on the same day = violation
            if day1 == day2:
                count_b2b_violation += 1
                print(f"shift:{reward_step},employee:{i}, constraint1 violated")
            
            # if shifts are on successive days, evening -> morning = violation
            if day2 == day1+1:
                # if shift 1 type = evening and shift 2 type = morning, record violation
                count_b2b_violation += [1 if shift_feats[:,4][0] == 1 and shift_feats[:,4][1] == 0 else 0][0]
                if [1 if shift_feats[:,4][0] == 1 and shift_feats[:,4][1] == 0 else 0][0] == 1:
                    print(f"shift:{reward_step},employee:{i}, constraint2 violated")

    return count_b2b_violation

In [343]:
print(num_shifts)
print(count_workers)
print(shift_features)

7
4
6


In [345]:
state = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
                  [1, 1, 0, 0, 0, 0, 0, 0, 0, 1],
                  [2, 1, 0, 0, 0, 1, 0, 0, 1, 0],
                  [3, 0, 1, 0, 0, 0, 0, 0, 1, 0],
                  [4, 0, 1, 0, 0, 1, 0, 1, 0, 0],
                  [5, 0, 0, 1, 0, 0, 0, 1, 0, 0],
                  [6, 0, 0, 0, 1, 0, 0, 1, 0, 0]])

In [356]:
for i in range(num_shifts):
    if i > 0:
        evaluateStep(state, reward_step=i)


shift:1,employee:3, constraint1 violated
shift:3,employee:2, constraint1 violated
shift:5,employee:1, constraint1 violated
shift:6,employee:1, constraint1 violated


In [357]:
evaluateSchedule(state)

shift:6,employee:1, constraint1 violated
shift:5,employee:1, constraint1 violated
shift:3,employee:2, constraint1 violated
shift:1,employee:3, constraint1 violated


4

In [358]:
v = evaluateSchedule(state)
v

shift:6,employee:1, constraint1 violated
shift:5,employee:1, constraint1 violated
shift:3,employee:2, constraint1 violated
shift:1,employee:3, constraint1 violated


4

In [ ]:
##### rewards

In [363]:
1 - (v / num_shifts)

0.4285714285714286

In [364]:
v

4